# 파라미터 갱신 파트 구현

파라미터 갱신 파트를 'train_network.py' 파일에 구현한다.

학습 횟수 값을 저장한 파라미터를 준비한다.

* 학습 데이터 로딩

'load_data()'는 셀프 플레이 파트에서 저장한 학습 데이터를 로드한다.

'with history_path.open(mode='rb') as f:'로 파일을 열고, 'pickle.load(f)'로 파이썬 객체를 복원한다.

* 듀얼 네트워크 학습

'train()'으로 듀얼 네트워크 학습을 수행한다.

1. 학습 데이터 로드

학습 데이터를 로드한다. 로드한 학습 데이터의 형태는 아래와 같다.

[[[자신의 돌의 위치, 상대방의 돌의 위치], 정책, 가치],
 [[자신의 돌의 위치, 상대방의 돌의 위치], 정책, 가치],
 [[자신의 돌의 위치, 상대방의 돌의 위치], 정책, 가치],
 ...]

 이를 zip()을 사용해서 상태, 정책, 가치별 리스트로 변환한다.

 [[자신의 돌의 위치, 상대방의 돌의 위치], [자신의 돌의 위치, 상대방의 돌의 위치], ...]

 [정책, 정책, ...]

 [가치, 가치, ...]

2. 학습을 위한 입력 데이터 형태 변환

듀얼 네트워크 생성에서 듀얼 네트워크의 입력 형태는 (3, 3, 2)로 했다. 그리고 학습, 평가, 추론에는 여러 입력 데이터를 모아서 전달할 수 있다. 이를 위해 학습, 평가, 추론에 전달하는 입력 데이터 형태는 입력 데이터 수의 차원을 더해 (입력 데이터 수, 3, 3, 2)가 된다.

여기서는 500개의 입력 데이터로 추론을 수행하므로 상태 리스트([[[자신의 돌의 위치, 상대방의 돌의 위치],[자신의 돌의 위치, 상대방의 돌의 위치], ...])를 형태 (500, 3, 3, 2)로 변환한다.

변환 순서는 아래와 같다.

 (1) 상태 리스트를 np.array()를 사용해 ndarray로 변환 

 (2) reshape()를 사용해 형태 (500, 2, 3, 3)으로 변환

 (3) transpose()를 사용해 축 선서를 바꾸어 형태 (3, 3, 2)로 변환

3. 베스트 플레이어 모델 로드

베스트 플레이어 모델을 로드한다. 최신 플레이어로서 학습시킨다.

듀얼 네트워크 생성에서 생성한 모델은 아직 학습이 되지 않은 상태이므로 최초에는 이 모델을 베스트 플레이어로 사용하고 학습 후 최신 플레이어의 모델을 저장한다.

4. 모델 컴파일

모델을 컴파일 한다. 여기서 정책은 분류이므로 'categorical_crossentropy', 가치는 회귀이므로 'mse'로 최적화 함수는 'Adam'으로 지정한다.

5. 학습률

학습률은 0.001에서 시작해서 50 스텝 이후 0.0005, 80 스텝 이후 0.00025로 떨어뜨린다.

6. 출력

콜백을 사용해 1 게임마다 경과를 출력하도록 한다.

7. 학습실행

8. 최신 플레이어 모델 저장

학습한 모델을 최신 플레이어 모델로 저장한다.

9. 모델 삭제

모델의 세션과 메모리를 파기한다.

---

* 오리지널 알파제로의 최적화 함수

오리지널 알파제로의 최적화 함수는 'SGD'를 사용한다. 속도를 위해 'Adam'을 선택했다.

* 오리지널 알파제로의 학습률

오리지널 알파제로 바둑의 학습률은 0.02에서 시작해 300스텝 후 0.002, 500,000 스텝 후 0.0002로 떨어뜨린다. 체스와 장의 학습률은 0.2에서 시작해 100 스텝 후 0.02, 300 스텝 후 0.002, 500,000 스텝 후 0.0002로 떨어뜨린다.

---

In [1]:
from google.colab import files
uploaded = files.upload()

# 폴더 확인
!dir

Saving dual_network.py to dual_network.py
Saving game.py to game.py
Saving pv_mcts.py to pv_mcts.py
Saving self_play.py to self_play.py
Saving train_network.py to train_network.py
dual_network.py  pv_mcts.py   self_play.py
game.py		 sample_data  train_network.py


In [2]:
!python game.py
!python dual_network.py
!python pv_mcts.py
!python self_play.py
!python train_network.py

---
---
-o-

--x
---
-o-

--x
-o-
-o-

x-x
-o-
-o-

x-x
-o-
-oo

x-x
-o-
xoo

xox
-o-
xoo

2021-04-15 16:37:06.479230: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-15 16:37:08.495948: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-15 16:37:08.496992: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-15 16:37:08.571460: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-15 16:37:08.572249: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 2

In [3]:
!dir ./model/

best.h5  lastet.h5
